In [ ]:
import setup_django
setup_django.init()

In [ ]:
import pandas as pd
from django.conf import settings
from movies.models import Movie
from ratings.models import Rating


LINKS_SMALL_CSV = settings.DATA_DIR / 'links_small.csv'
LINKS_SMALL_CSV.exists()

In [ ]:
qs = Rating.objects.all()
missing_movie_ids = []
for instance in qs:
    if instance.content_object is None:
        missing_movie_ids.append(instance.object_id)

        
_total = len(missing_movie_ids)
total_missing = list(set(missing_movie_ids))

In [ ]:
print(len(total_missing), _total, qs.count())

In [ ]:
links_df = pd.read_csv(LINKS_SMALL_CSV)
links_df.head()

In [ ]:
ms_df = links_df.copy()[links_df.movieId.isin(total_missing)]
ms_df.head()





In [ ]:
ms_df.shape[0] == len(total_missing)

In [ ]:
def enrich_imdb_col(val):
    val = str(val)
    if len(val) == 7:
        val = f"tt{val}"
        return val
    if len(val) == 6:
        val = f"tt0{val}"
        return val
    if len(val) == 5:
        val = f"tt00{val}"
        return val
    return val

In [ ]:
ms_df['tt'] = ms_df['imdbId'].apply(enrich_imdb_col)
# ms_df.to_csv('')
ms_df.head()

In [ ]:
MOVIES_CSV = settings.DATA_DIR / 'movies_metadata.csv'
MOVIES_CSV.exists()

In [ ]:
movies_cols = ['title', 'overview', 'release_date', 'imdb_id']
movies_df = pd.read_csv(MOVIES_CSV, usecols=movies_cols)
movies_df.head()

In [ ]:
missing_movies_df = ms_df.merge(movies_df, left_on='tt', right_on='imdb_id')
missing_movies_df.head()

In [ ]:
missing_movies_df['id'] = missing_movies_df['movieId']
missing_movies_df['id_alt'] = missing_movies_df['tmdbId'].apply(lambda x: str(int(x)))
missing_movies_df.head()

In [ ]:
final_df = missing_movies_df.copy()[['id', 'id_alt', 'title']]
final_df['id_alt'] = final_df['id_alt'].astype(str)
final_df.head()

In [ ]:
alt_id_list = final_df['id_alt'].to_list()

In [ ]:

movies_qs.count()

In [ ]:
from django.forms.models import model_to_dict

In [ ]:
movies_qs = Movie.objects.filter(id__in=alt_id_list)
for obj in movies_qs:
    data = final_df.copy()[final_df['id_alt'] == str(obj.id)]
    if data.shape[0] == 1:
        og_model_data = model_to_dict(obj)
        update_data = data.to_dict('records')[0]
        if obj.title == update_data.get('title'): 
            # print(og_model_data)
            og_model_data['id'] = update_data['id']
            new_model_data = {**og_model_data}
            # print(new_model_data)
            obj.delete()
            Movie.objects.create(**new_model_data)

In [ ]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()